## **Przykłady: Wieloagentowe AI do rezerwacji hotelu**

We współczesnym, szybkim świecie planowanie podróży służbowej to coś więcej niż tylko rezerwacja lotu i pokoju hotelowego. Wymaga to poziomu koordynacji i efektywności, który może być trudny do osiągnięcia. Tutaj właśnie wkraczają w grę Wieloagentowe AI, rewolucjonizujące sposób zarządzania naszymi potrzebami podróżnymi.

Wyobraź sobie, że masz do dyspozycji zespół inteligentnych agentów, którzy współpracują, aby z precyzją i łatwością zająć się każdym aspektem twojej podróży. Dzięki naszej zaawansowanej technologii AI stworzyliśmy wyspecjalizowanych agentów do rezerwacji usług i organizacji planów podróży, zapewniając płynne i pozbawione stresu doświadczenie podróżne.

To jest podstawowy scenariusz. Podczas planowania podróży służbowej musimy skonsultować się z agentem podróży służbowej, aby uzyskać informacje o biletach lotniczych, hotelach itp. Poprzez Agentów AI możemy zbudować agentów do rezerwacji usług oraz agentów do organizacji planu podróży, którzy będą współpracować i podnosić poziom inteligencji.


# Inicjalizacja usługi Azure AI Agent oraz pobranie informacji konfiguracyjnych z **.env**

### **.env**

Utwórz plik .env

**.env** zawiera ciąg połączenia usługi Azure AI Agent Service, model używany przez AOAI oraz odpowiadające temu usłudze Google API Search API, ENDPOINT itp.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nazwa wdrożenia modelu w usłudze Azure AI Agent Service"

[**NOTE**] Potrzebny będzie model z limitem 100 000 Rate Limit (Tokenów na minutę) oraz limitem 600 (Żądań na minutę)

  Model można uzyskać w Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Twój ciąg połączenia projektu usługi Azure AI Agent Service"

  Ciąg połączenia projektu można znaleźć w przeglądzie projektu na ekranie portalu AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Twoj klucz API SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Twój punkt końcowy SERPAPI Search"

Aby uzyskać nazwę wdrożenia modelu i ciąg połączenia projektu usługi Azure AI Agent Service, musisz utworzyć Azure AI Agent Service. Zaleca się użycie [tego szablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json), aby utworzyć ją bezpośrednio (***Uwaga:*** Usługa Azure AI Agent Service jest obecnie ustawiona w ograniczonym regionie. Zaleca się, aby odnieść się do [tego linku](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support), aby ustawić region)

Agent potrzebuje dostępu do SERPAPI. Zaleca się rejestrację przy użyciu [tego linku](https://serpapi.com/searches). Po rejestracji można uzyskać unikalny klucz API (API KEY) oraz punkt końcowy (ENDPOINT).


# Logowanie do Azure

Teraz musisz zalogować się do Azure. Otwórz terminal w VScode i uruchom polecenie `az login`


# Setup 

Aby uruchomić ten notatnik, musisz zainstalować następujące biblioteki. Oto lista wymaganych bibliotek oraz odpowiadające im polecenia pip install:

azure-identity: Do uwierzytelniania w Azure.  
requests: Do wykonywania zapytań HTTP.  
semantic-kernel: Do frameworka semantic kernel (zakładając, że jest to niestandardowa lub konkretna biblioteka, może być konieczne zainstalowanie jej z określonego źródła lub repozytorium).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Wyjaśnienie: 
import asyncio: Importuje moduł asyncio, który zapewnia wsparcie dla programowania asynchronicznego w Pythonie. Umożliwia pisanie współbieżnego kodu z użyciem składni async i await.
from typing import Annotated: Importuje typ Annotated z modułu typing. Annotated służy do dodawania metadanych do wskazówek typów, co może być użyteczne do różnych celów, takich jak walidacja, dokumentacja czy narzędzia.


In [ ]:
import asyncio,os
from typing import Annotated

# Wyjaśnienie:
Używając from dotenv import load_dotenv i load_dotenv(), możesz łatwo zarządzać ustawieniami konfiguracyjnymi oraz poufnymi informacjami (takimi jak klucze API i adresy URL baz danych) w pliku .env, oddzielając je od kodu źródłowego i czyniąc aplikację bardziej bezpieczną oraz łatwiejszą do skonfigurowania.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Wyjaśnienie:

Importowanie: from azure.identity.aio import DefaultAzureCredential: Importuje klasę DefaultAzureCredential z modułu azure.identity.aio. Część aio w nazwie modułu wskazuje, że jest on przeznaczony do operacji asynchronicznych.

Cel DefaultAzureCredential: Klasa DefaultAzureCredential jest częścią Azure SDK dla Pythona. Zapewnia domyślny sposób uwierzytelniania się w usługach Azure. Próbuje uwierzytelnić się za pomocą wielu metod w określonej kolejności, takich jak zmienne środowiskowe, zarządzana tożsamość oraz poświadczenia Azure CLI.

Operacje asynchroniczne: Moduł aio wskazuje, że klasa DefaultAzureCredential obsługuje operacje asynchroniczne. Oznacza to, że można jej używać z asyncio do wykonywania nieblokujących żądań uwierzytelniania.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Wyjaśnienie:
Importuje różne moduły i klasy z pakietu semantic_kernel. Oto rozbicie poszczególnych importów:

AgentGroupChat z semantic_kernel.agents: Ta klasa obsługuje funkcjonalności związane z czatem grupowym dla agentów AI. AzureAIAgent i AzureAIAgentSettings z semantic_kernel.agents.azure_ai

AzureAIAgent: Ta klasa służy do tworzenia i zarządzania agentami AI, którzy korzystają z usług Azure AI.

AzureAIAgentSettings: Ta klasa służy do konfigurowania ustawień dla AzureAIAgent. TerminationStrategy z semantic_kernel.agents.strategies.termination.termination_strategy:

Ta klasa definiuje strategie zakończenia działania agentów AI pod pewnymi warunkami. ChatMessageContent z semantic_kernel.contents.chat_message_content:

Ta klasa służy do obsługi treści wiadomości czatu.
AuthorRole z semantic_kernel.contents.utils.author_role:

Ta klasa definiuje różne role autorów w kontekście wiadomości czatu.

kernel_function z semantic_kernel.functions.kernel_function_decorator: Ten dekorator służy do definiowania funkcji jądra, które mogą być wykonywane w ramach struktury semantic kernel.
Te importy przygotowują niezbędne komponenty do tworzenia i zarządzania agentami AI, którzy mogą wchodzić w interakcje w środowisku czatu grupowego, prawdopodobnie do zadań takich jak rezerwacja hoteli lub podobne działania.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:
Następnie importujemy klasę CodeInterpreterTool z modułu azure.ai.projects.models. 

CodeInterpreterTool: Ta klasa jest częścią Azure AI SDK i jest używana do interpretowania i wykonywania kodu w kontekście projektów AI. Zapewnia funkcjonalności do uruchamiania fragmentów kodu, analizy kodu lub integracji wykonania kodu w ramach przepływów pracy AI.
Ten import przygotowuje niezbędny komponent do wykorzystania CodeInterpreterTool w Twoim projekcie, co może być przydatne do zadań związanych z dynamicznym interpretowaniem i wykonywaniem kodu.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Wyjaśnienie: 
Klasa ApprovalTerminationStrategy zapewnia konkretną strategię zakończenia działania agenta AI. Agent zakończy działanie, jeśli ostatnia wiadomość w jego historii interakcji zawiera słowo „saved”. Może to być użyteczne w scenariuszach, w których zadanie agenta jest uznawane za zakończone po otrzymaniu potwierdzenia, że coś zostało „saved”.Zdefiniuj metodę interakcji. Po zapisaniu planu rezerwacji, można go zatrzymać po otrzymaniu sygnału saved


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Wyjaśnienie:

Linia kodu inicjalizuje obiekt AzureAIAgentSettings z domyślnymi lub predefiniowanymi ustawieniami, wywołując metodę create(). Ten obiekt ustawień (ai_agent_settings) może być następnie użyty do konfiguracji i zarządzania instancją AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Wyjaśnienie:
Importując bibliotekę requests, można łatwo wykonywać żądania HTTP oraz współdziałać z usługami internetowymi w swoim kodzie Pythona.


In [ ]:
import requests

# Explanation:
To jest zmienna, która przechowuje klucz API do uzyskania dostępu do usługi API SERP (Search Engine Results Page). Klucz API to unikalny identyfikator używany do uwierzytelniania żądań powiązanych z Twoim kontem.

'GOOGLE_SEARCH_API_KEY': To jest przykładowy ciąg znaków. Musisz zastąpić ''GOOGLE_SEARCH_API_KEY' swoim rzeczywistym kluczem API SERP.

Cel: Celem tej linii jest przechowywanie klucza API w zmiennej, aby można go było użyć do uwierzytelniania żądań do usługi API SERP. Klucz API jest wymagany do uzyskania dostępu do usługi i wykonywania wyszukiwań.

Jak uzyskać klucz API SERP: Aby uzyskać klucz API SERP, wykonaj następujące ogólne kroki na https://serpapi.com (konkretne kroki mogą się różnić w zależności od używanej usługi API SERP):

Wybierz usługę API SERP: Istnieje kilka dostępnych usług API SERP, takich jak SerpAPI, Google Custom Search JSON API i inne. Wybierz tę, która najlepiej odpowiada Twoim potrzebom.

Zarejestruj konto:

Przejdź na stronę wybranej usługi API SERP https://www.serpapi.com i zarejestruj konto. Może być konieczne podanie podstawowych informacji i potwierdzenie adresu e-mail.

Utwórz klucz API:

Po rejestracji zaloguj się na swoje konto i przejdź do sekcji API lub panelu kontrolnego. Poszukaj opcji utworzenia lub wygenerowania nowego klucza API.
Skopiuj klucz API:

Gdy klucz API zostanie wygenerowany, skopiuj go. Ten klucz będzie używany do uwierzytelniania Twoich żądań do usługi API SERP.
Zamień symbol zastępczy:

Zamień symbol zastępczy w swoim pliku .env


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Wyjaśnienie:
Klasa BookingPlugin udostępnia metody do rezerwacji hoteli i lotów za pomocą API wyszukiwania Google Serpapi.com. Buduje niezbędne parametry, wysyła żądania do API oraz przetwarza odpowiedzi, aby zwrócić odpowiednie informacje o rezerwacji. Klucz API (SERPAPI_SEARCH_API_KEY) oraz punkt końcowy (SERPAPI_SEARCH_ENDPOINT) służą do uwierzytelniania i wysyłania żądań do API wyszukiwania Google.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Wyjaśnienie:
Klasa SavePlugin udostępnia metodę saving_plan do zapisywania planów podróży za pomocą usług Azure AI. Ustawia poświadczenia Azure, tworzy agenta AI, przetwarza dane wejściowe użytkownika w celu wygenerowania i zapisania zawartości planu podróży oraz obsługuje zapisywanie plików i operacje porządkowe. Metoda zwraca "Saved" po pomyślnym zakończeniu.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Explanation:
Ten kod konfiguruje agentów Azure AI do obsługi rezerwacji lotów i hoteli oraz zapisywania planów podróży na podstawie danych wprowadzonych przez użytkownika. Używa poświadczeń Azure do tworzenia i konfigurowania agentów, przetwarza dane wejściowe użytkownika za pomocą czatu grupowego i zapewnia odpowiednie sprzątanie po wykonaniu zadań. Agenci korzystają ze specyficznych wtyczek (BookingPlugin i SavePlugin) do realizacji swoich zadań.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Niniejszy dokument został przetłumaczony przy użyciu usługi tłumaczeń AI [Co-op Translator](https://github.com/Azure/co-op-translator). Dokładamy starań, aby tłumaczenie było jak najdokładniejsze, jednak prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub niedokładności. Oryginalny dokument w jego języku źródłowym powinien być traktowany jako źródło autorytatywne. W przypadku informacji o krytycznym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
